In [4]:
from parse import *

# File format
The current implementation reads in a file (or accepts a string) and looks for %HTCSS as a special marker. It will then add the text between markers to the specified field of a dictionary (TEMPLATE or TABLE). These are the only section headers it does anything with, and both are required.

The pattern and parsing are flexible and this is just the first implementation.



In [5]:
FILE = """
%HTCSS TEMPLATE
executable=dummy.sh
arguments= $(a) $(b)
output=cat-$(ProcId).out
error=cat-$(ProcId).err
log=cat.log
request_cpus=1
request_memory=128MB
request_disk=128MB
queue from TABLE table.csv
%HTCSS TABLE
a,b
1,2
3,4
5,6
"""

One current pain point is that the QUEUE FROM TABLE necessitates reading from a file on disk, so the parsed table gets written out (and optionally cleaned up)

In [7]:
res = parse_htcss_string(FILE)
print("-----")
print("Table: ")
print(res["TABLE"])
print("-----")
print("Submit template: ")
print(res["TEMPLATE"])

-----
Table: 
a,b
1,2
3,4
5,6
-----
Submit template: 
executable=dummy.sh
arguments= $(a) $(b)
output=cat-$(ProcId).out
error=cat-$(ProcId).err
log=cat.log
request_cpus=1
request_memory=128MB
request_disk=128MB
queue from TABLE table.csv


# Job submission
The parsed template is a valid Submit object within the python bindings, so job submission is the same as a typical binding-driven one:

In [8]:
schedd = htcondor.Schedd()
submit_result = schedd.submit(htcondor.Submit(res["TEMPLATE"]))

HTCondorException: Unable to locate local daemon.